In [1]:
import os
import numpy as np
from nltk.corpus import stopwords
from string import punctuation
from gensim.models import word2vec

In [2]:
os.getcwd()

'C:\\Users\\Simon\\packt_video_courses\\Natural-Language-Processing-with-Real-World-Projects\\Section 1 - 11 Resources - NLP-Code\\Introduction to Semantic Processing\\Introduction'

### Loading the training and test data sets

In [3]:
train_data = "attachment_r8-train-all-terms_lyst3317.txt"
test_data = "attachment_r8-test-all-terms_lyst1822.txt"

In [4]:
X, y = [], []
with open(train_data, "r") as infile:
    for line in infile:
        label, text = line.split("\t")
        X.append(text.split())
        y.append(label)
print("total examples %s" % len(y))
                

total examples 5485


In [5]:
X_test, y_test = [], []
with open(test_data, "r") as infile:
    for line in infile:
        label, text = line.split("\t")
        X_test.append(text.split())
        y_test.append(label)
print("total examples %s" % len(y_test))   

total examples 2189


In [6]:
np.unique(y, return_counts=True)

(array(['acq', 'crude', 'earn', 'grain', 'interest', 'money-fx', 'ship',
        'trade'], dtype='<U8'),
 array([1596,  253, 2840,   41,  190,  206,  108,  251], dtype=int64))

### Using NB methods

In [7]:
X_text = [" ".join(val) for val in X]
print(X_text[0] + "\n")
X_test_text = [" ".join(val) for val in X_test]
print(X_test_text[10])

champion products ch approves stock split champion products inc said its board of directors approved a two for one stock split of its common shares for shareholders of record as of april the company also said its board voted to recommend to shareholders at the annual meeting april an increase in the authorized capital stock from five mln to mln shares reuter

vieille montagne reports loss dividend nil year net loss after exceptional charges mln francs vs profit mln exceptional provisions for closure of viviez electrolysis plant mln francs vs exceptional gain mln sales and services billion francs vs billion proposed net dividend on ordinary shares nil vs francs company s full name is vieille montagne sa vmnb br reuter


#### Using count vectorizer

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
vect = CountVectorizer(stop_words="english", max_features=5000)
vect.fit(X_text)

CountVectorizer(max_features=5000, stop_words='english')

Applying the vectorizer to our test and train sets

In [10]:
X_train_transformed = vect.transform(X_text)
X_test_transformed = vect.transform(X_test_text)

In [11]:
list(vect.vocabulary_.items())[:10]

[('champion', 759),
 ('products', 3488),
 ('approves', 264),
 ('stock', 4337),
 ('split', 4273),
 ('said', 3974),
 ('board', 532),
 ('directors', 1327),
 ('approved', 263),
 ('common', 895)]

### Using Bernoulli NB first

In [12]:
from sklearn.metrics import accuracy_score

In [13]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
bnb.fit(X_train_transformed, y)

#predict class
pred_train_ys = bnb.predict(X_train_transformed)
pred_test_ys = bnb.predict(X_test_transformed)

#accuracy
print("Train accuracy: ", accuracy_score(y, pred_train_ys))
print("Train accuracy: ", accuracy_score(y_test, pred_test_ys))

Train accuracy:  0.8736554238833182
Train accuracy:  0.8688899040657835


### Using Multinomial NB
- We expect this to work very well, giving high performance in accuracy

In [14]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

In [15]:
#fit on training data
mnb.fit(X_train_transformed, y)

#predict class
pred_train_ys = mnb.predict(X_train_transformed)
pred_test_ys = mnb.predict(X_test_transformed)

#accuracy
print("Train accuracy: ", accuracy_score(y, pred_train_ys))
print("Train accuracy: ", accuracy_score(y_test, pred_test_ys))

Train accuracy:  0.968094804010939
Train accuracy:  0.9657377798081316


- as expected, this performed really well
- remebmer that we used 5000 features!

### Using our word embeddings approach

We have two option here:
1. Use pre-trained word vectors (Glove)
2. Train our own vectors

We'll explore both

### Loading the pre-trained GloVe word vectors

In [17]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

glove_input_file = "F:\Downloads\glove.6B\glove.6B.200d.txt"
word2vec_output_file = "F:\Downloads\glove.6B\glove.6B.200d.w2vformat.txt"
glove2word2vec(glove_input_file, word2vec_output_file)

C:\Users\Simon\AppData\Local\Temp\ipykernel_15616\3826241164.py:6: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


(400000, 200)

In [18]:
from gensim.models.keyedvectors import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format(glove_input_file, binary=False, no_header=True)

### Sentence vector by averaging word vectors

In [19]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Simon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
stop_words = stopwords.words("english")

In [21]:
def sent_vec(sent):
    wv_res = np.zeros(glove_model.vector_size)
    ctr = 1
    for w in sent:
        if w in glove_model:
            ctr += 1    
            wv_res += glove_model[w]
    wv_res = wv_res/ctr
    return wv_res    

In [22]:
train_doc_vecs = []
for doc in X:
    doc_words = [term for term in doc if term not in stop_words]
    train_doc_vecs.append(sent_vec(doc_words))

In [23]:
test_doc_vecs = []
for doc in X_test:
    doc_words = [term for term in doc if term not in stop_words]
    test_doc_vecs.append(sent_vec(doc_words))

### Building a predictive model on the averaged word vectors


#### Using a "simple" logistic regression

In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
logreg = LogisticRegression(penalty="l2", random_state=42, C=3.5)
logreg.fit(train_doc_vecs, y)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=3.5, random_state=42)

In [26]:
pred_train_ys = logreg.predict(train_doc_vecs)
pred_test_ys = logreg.predict(test_doc_vecs)
print("Train accuracy: ", accuracy_score(pred_train_ys, y))
print("Test accuracy: ", accuracy_score(pred_test_ys, y_test))

Train accuracy:  0.9797629899726527
Test accuracy:  0.9680219278209228


### Training our own word vectors on the data
We'll create a combined text file to train our word vectors - more data is better. Although in this case we would still have just 7.7K instances to learn from

In [31]:
X_comb = np.concatenate([X, X_test])
#len(X_comb)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5485,) + inhomogeneous part.

In [29]:
print(X_comb[6000])

NameError: name 'X_comb' is not defined

In [ ]:
w2v = word2vec.Word2Vec(X_comb, window=2, min_count=2, sg=1, size=200)

In [ ]:
w2v.most_similar("future", topn=5)

####  Sentence vectors by averaging vectors for words

In [ ]:
def sent_vec_w2v(sent):
    wv_res = np.zeros(w2v.vector_size)
    ctr = 1
    for w in sent:
        if w in w2v:
            ctr+=1
            wv_res += w2v[w]
    wv_res = wv_res/ctr
    return wv_res

#### Getting the sentence vectors for the test and train sets

In [ ]:
train_doc_vecs = []
for doc in X:
    doc_words = [term for term in doc if term not in stop_words]
    train_doc_vecs.append(sent_voc_w2v(doc_words))

In [ ]:
test_doc_vecs = []
for doc in X_test:
    doc_words = [term for term in doc if term not in stop_words]
    test_doc_vecs.append(sent_voc_w2v(doc_words))

#### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression(penalty="l1", random_state=42, C = 8)

In [ ]:
logreg.fit(train_doc_vecs, y)

In [ ]:
pred_train_ys = logreg.predict(train_doc_vecs)
pred_test_ys = logreg.predict(test_doc_vecs)
print("Train accuracy: ", accuracy_score(pred_train_ys, y))
print("Test accuracy: ", accuracy_score(pred_test_ys, y))